In [1]:
import cv2
import numpy as np
import pylab
import os
import shutil
import imageio
from collections import deque
from collections import deque
import numpy as np
import argparse

import imutils
import cv2

In [2]:
try:
    os.mkdir("images")
except:
    shutil.rmtree("images")
    os.mkdir("images")

In [3]:
greenLower = (29, 86, 6)
greenUpper = (64, 255, 255)
 
# initialize the list of tracked points, the frame counter,
# and the coordinate deltas
pts = deque(maxlen=20)
counter = 0
(dX, dY) = (0, 0)
direction = ""
filename = 'test_sample1.avi'
vid = imageio.get_reader(filename,  'ffmpeg')
count = 0
for num in range(vid.get_length()):
    try:
        img = vid.get_data(num)
        
        img = cv2.resize(img,(1000,600))
        gray_image = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        _,im =  cv2.threshold(gray_image,220,1,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        
        image, cnts, hier = cv2.findContours(im,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        center = None
        for c in cnts:
            x, y, w, h = cv2.boundingRect(c)

            # draw a green rectangle to visualize the bounding rect
            cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)

            # get the min area rect
            rect = cv2.minAreaRect(c)
            box = cv2.boxPoints(rect)

            # convert all coordinates floating point values to int
            box = np.int0(box)

            # draw a red 'nghien' rectangle
            cv2.drawContours(img, [box], 0, (0, 0, 255))
        # only proceed if at least one contour was found
        
        if len(cnts) > 1:
            
            # Change objects for finding trajectory
            # By changing the value of -2 to any other integer you can change the tracking
            # if you change it to -3 it will pickup the object which has third largest area  
            # and so on. but the area of objects will change as you move and rotate the image
            # So it will not give much accurate results
            # -2 works best because it is the second largest area. first largest area is usually 
            # the full video size.
            c = sorted(cnts, key=cv2.contourArea)[-2]
            
            # for tracking most dominant object uncomment this and comment above  line
#             c = [contour for contour in sorted(cnts, key=cv2.contourArea)[::-1] if cv2.contourArea(contour)< 450000][0]
             
            
            ((x, y), radius) = cv2.minEnclosingCircle(c)
            M = cv2.moments(c)
            center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
            cv2.circle(img, (int(x), int(y)), int(radius),
                (0, 255, 255), 2)
            cv2.circle(img, center, 5, (0, 0, 255), -1)
            pts.appendleft(center)
            
        # loop over the set of tracked points
        for i in np.arange(1, len(pts)):
            # if either of the tracked points are None, ignore
            # them
            if pts[i - 1] is None or pts[i] is None:
                continue
                # check to see if enough points have been accumulated in
                # the buffer
            if counter >= 10 and i == 1 and pts[-10] is not None:
                # compute the difference between the x and y
                # coordinates and re-initialize the direction
                # text variables
                dX = pts[-10][0] - pts[i][0]
                dY = pts[-10][1] - pts[i][1]
                (dirX, dirY) = ("", "")

                # ensure there is significant movement in the
                # x-direction
                if np.abs(dX) > 20:
                    dirX = "East" if np.sign(dX) == 1 else "West"

                # ensure there is significant movement in the
                # y-direction
                if np.abs(dY) > 20:
                    dirY = "North" if np.sign(dY) == 1 else "South"

                # handle when both directions are non-empty
                if dirX != "" and dirY != "":
                    direction = "{}-{}".format(dirY, dirX)

                # otherwise, only one direction is non-empty
                else:
                    direction = dirX if dirX != "" else dirY
            # otherwise, compute the thickness of the line and
            # draw the connecting lines
            thickness = int(np.sqrt(15 / float(i + 1)) * 2.5)
            cv2.line(img, pts[i - 1], pts[i], (0, 0, 255), thickness)

        # show the movement deltas and the direction of movement on
        # the frame
    #         cv2.drawContours(img, cnts[1], -1, (255, 255, 0), 1)
        cv2.putText(img, direction, (10, 30), cv2.FONT_HERSHEY_SIMPLEX,
            0.65, (0, 0, 255), 3)
        cv2.putText(img, "dx: {}, dy: {}".format(dX, dY),
            (10, img.shape[0] - 10), cv2.FONT_HERSHEY_SIMPLEX,
            0.35, (0, 0, 255), 1)

        # show the frame to our screen and increment the frame counter
        cv2.imshow("Frame", img)
        count += 1
        cv2.imwrite("images/"+"image-"+str(count)+".jpg",img)
        key = cv2.waitKey(1) & 0xFF
        counter += 1

        # if the 'q' key is pressed, stop the loop
        if key == ord("q"):
            break

    except:
        pass
# cleanup the camera and close any open windows
# camera.release()
cv2.destroyAllWindows()

In [4]:
try:
    %rm -rf output-vid.avi
except:
    pass

In [5]:
!ffmpeg -i images/image-%00d.jpg -r 76 -s 1280x720 output-vid.avi

ffmpeg version 2.7 Copyright (c) 2000-2015 the FFmpeg developers
  built with llvm-gcc 4.2.1 (LLVM build 2336.11.00)
  configuration: --prefix=/Volumes/Ramdisk/sw --enable-gpl --enable-pthreads --enable-version3 --enable-libspeex --enable-libvpx --disable-decoder=libvpx --enable-libmp3lame --enable-libtheora --enable-libvorbis --enable-libx264 --enable-avfilter --enable-libopencore_amrwb --enable-libopencore_amrnb --enable-filters --enable-libgsm --enable-libvidstab --enable-libx265 --disable-doc --arch=x86_64 --enable-runtime-cpudetect
  libavutil      54. 27.100 / 54. 27.100
  libavcodec     56. 41.100 / 56. 41.100
  libavformat    56. 36.100 / 56. 36.100
  libavdevice    56.  4.100 / 56.  4.100
  libavfilter     5. 16.101 /  5. 16.101
  libswscale      3.  1.101 /  3.  1.101
  libswresample   1.  2.100 /  1.  2.100
  libpostproc    53.  3.100 / 53.  3.100
Input #0, image2, from 'images/image-%00d.jpg':
  Duration: 00:00:23.56, start: 0.000000, bitrate: N/A
    Stream #0:0: Video: mj

In [6]:
c = sorted(cnts, key=cv2.contourArea)[::-1][0]


In [7]:
cv2.contourArea(c)

595087.5

In [8]:
img = cv2.resize(img,(1000,600))

In [9]:
img.shape

(600, 1000, 3)